In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from IPython.display import clear_output 

!pip install transformers==4.8.2
!pip install sentencepiece==0.1.96 # 이번 실습에서 추가되었습니다
!pip install tensorflow_addons

clear_output() # clear the output after the installation

In [7]:
import sklearn
import tensorflow
import transformers
import tensorflow_addons

print(sklearn.__version__) # 1.0.2
print(tensorflow.__version__) # 2.8.0
print(transformers.__version__) # 4.8.2
print(tensorflow_addons.__version__) # 0.16.1

# import sentencepiece # 이번 실습에서 추가되었습니다
# !pip show sentencepiece # 0.1.96 (이번 실습에서 추가되었습니다)

1.0.2
2.8.0
4.8.2
0.17.0


In [8]:
import pandas as pd
import numpy as np

import os
import re
import pickle 
import dill # for saving a function as a file(.pkl)
import logging # for changing the tf's logging level
import urllib.request
from tqdm import tqdm

from sklearn import model_selection
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow_addons as tfa # for using Rectified-Adam optimizer (instead of Adam optimizer) 
from tensorflow.keras import layers, initializers, losses, optimizers, metrics, callbacks 

import transformers
from transformers import TFBertModel # BertTokenizer 제외

import sentencepiece as spm # 이번 실습에서 추가되었습니다

In [9]:
# Random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

# Transformers logging level 변경 (WARNING -> ERROR) @ https://huggingface.co/transformers/main_classes/logging.html
transformers.logging.set_verbosity(transformers.logging.ERROR)

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # 원하는 세부 경로로 변경이 가능합니다.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [17]:
data_df = pd.read_csv('./drive/MyDrive/data_df.csv', index_col=0)

In [19]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(data_df['Sentence'], data_df['Label'], 
                                                                    test_size=0.3, 
                                                                    random_state=42) 

In [20]:
!git clone https://github.com/monologg/KoBERT-Transformers.git
!mv KoBERT-Transformers/kobert_transformers/tokenization_kobert.py /content
clear_output() # clear the output

In [21]:
from tokenization_kobert import KoBertTokenizer 

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert') # sentencepiece 라이브러리가 먼저 import 되어있어야 합니다.
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', cache_dir='bert_ckpt', do_lower_case=False) 

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

In [22]:
SEQ_LEN = 128 # 최대 token 개수 이상의 값으로 임의로 설정

token_ids =[]
token_masks =[]
token_segments =[]

train_labels = []


for idx in tqdm(range(len(train_x))):

    train_sentence = train_x.iloc[idx]
    
    # 특수문자 제거
    cleaned_sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", train_sentence) # [ whitespaces, 숫자, 영문 알파벳, 한글(+자모음) ]이 아닌 것을 공백으로 치환 (특수문자 제거)

    # Tokenizing / Tokens to sequence numbers / Padding
    encoded_dict = tokenizer.encode_plus(text=cleaned_sentence,
                                         padding='max_length', 
                                         truncation = True,
                                         max_length=SEQ_LEN) # SEQ_LEN == 512
    
    token_ids.append(encoded_dict['input_ids']) # tokens_tensor
    token_masks.append(encoded_dict['attention_mask']) # masks_tensor
    token_segments.append(encoded_dict['token_type_ids']) # segments_tensor

    train_labels.append(train_y.iloc[idx])
        

train_inputs = (np.array(token_ids), np.array(token_masks), np.array(token_segments))
train_labels = np.array(train_labels)

100%|██████████| 78637/78637 [00:18<00:00, 4187.68it/s]


In [23]:
def create_model(max_length=128):

    bert_base_model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True) 
    # bert_base_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

    input_token_ids   = layers.Input((max_length,), dtype=tf.int32, name='input_token_ids')   # tokens_tensor
    input_masks       = layers.Input((max_length,), dtype=tf.int32, name='input_masks')       # masks_tensor
    input_segments    = layers.Input((max_length,), dtype=tf.int32, name='input_segments')    # segments_tensor  

    bert_outputs = bert_base_model([input_token_ids, input_masks, input_segments]) 
    # bert_outputs -> 0: 'last_hidden_state' & 1: 'pooler_output' (== applied GlobalAveragePooling1D on 'last_hidden_state')

    bert_outputs = bert_outputs[1] # ('pooler_output', <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model')>)
    bert_outputs = layers.Dropout(0.2)(bert_outputs)
    final_output = layers.Dense(units=3, activation='softmax', kernel_initializer=initializers.TruncatedNormal(stddev=0.02), name="classifier")(bert_outputs)

    model = tf.keras.Model(inputs=[input_token_ids, input_masks, input_segments], 
                        outputs=final_output)

    # RAdam (Rectified-Adam) @ http://j.mp/2P5OmF3 / http://j.mp/2qzRjUa / http://j.mp/2N322hu / http://j.mp/2MYtPQ2
    model.compile(optimizer=tfa.optimizers.RectifiedAdam(learning_rate=1e-5, weight_decay=0.0025, warmup_proportion=0.05),
                  loss=losses.SparseCategoricalCrossentropy(), 
                  metrics=[metrics.SparseCategoricalAccuracy()])
    
    return model

In [24]:
# TPU 세팅 (TPU 클러스터 할당 & 초기화)

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("\nAll devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.TPUStrategy(resolver) # Obsolete : tf.distribute.experimental.TPUStrategy()

with strategy.scope(): 
    model = create_model(max_length=128) # TPU Cluster 상에서 모델 생성 & Compile 진행

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0



All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [25]:
# Checkpoint setting for saving the best model
from google.colab import drive
drive.mount('/gdrive')

checkpoint_path = '/gdrive/MyDrive/colab_data/temp_data/saved_models/'

if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)


# For custom models, we have to use "save_weights_only = True" (or we should implement a "get_config" method @ https://j.mp/3ltUibd) 
callback_checkpoint = callbacks.ModelCheckpoint(filepath=checkpoint_path + 'best_kobert_weights.h5', # 이번 실습에서 변경되었습니다
                                                monitor='val_sparse_categorical_accuracy',
                                                save_best_only=True, 
                                                save_weights_only = True, 
                                                verbose=1) 
                                                
# Early-stopping for preventing the overfitting
callback_earlystop = callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', 
                                             min_delta=0.0001, # the threshold that triggers the termination (acc should at least improve 0.0001)
                                             patience=5) #  Number of epochs with no improvement after which training will be stopped

Mounted at /gdrive


In [26]:
# 25분 가량 소요됩니다 (epochs를 4~5 정도로 주더라도 비슷한 수준으로 학습이 가능합니다)

history = model.fit(train_inputs, train_labels, validation_split=0.2,
                    epochs=10, batch_size=100,
                    verbose=1,
                    callbacks=[callback_checkpoint, callback_earlystop])

Epoch 1/10


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None,) dtype=int64>]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None,) dtype=int64>]


630/630 [==============================] - ETA: 0s - loss: 0.8687 - sparse_categorical_accuracy: 0.5723

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None,) dtype=int64>]



Epoch 1: val_sparse_categorical_accuracy improved from -inf to 0.68750, saving model to /gdrive/MyDrive/colab_data/temp_data/saved_models/best_kobert_weights.h5
630/630 [==============================] - 253s 230ms/step - loss: 0.8687 - sparse_categorical_accuracy: 0.5723 - val_loss: 0.7044 - val_sparse_categorical_accuracy: 0.6875
Epoch 2/10
630/630 [==============================] - ETA: 0s - loss: 0.6726 - sparse_categorical_accuracy: 0.7051
Epoch 2: val_sparse_categorical_accuracy improved from 0.68750 to 0.72266, saving model to /gdrive/MyDrive/colab_data/temp_data/saved_models/best_kobert_weights.h5
630/630 [==============================] - 84s 134ms/step - loss: 0.6726 - sparse_categorical_accuracy: 0.7051 - val_loss: 0.6327 - val_sparse_categorical_accuracy: 0.7227
Epoch 3/10
630/630 [==============================] - ETA: 0s - loss: 0.6155 - sparse_categorical_accuracy: 0.7328
Epoch 3: val_sparse_categorical_accuracy improved from 0.72266 to 0.73220, saving model to /gdrive/

In [27]:
SEQ_LEN = 128 # 최대 token 개수 이상의 값으로 임의로 설정

token_ids =[]
token_masks =[]
token_segments =[]

test_labels = []


for idx in tqdm(range(len(test_x))):

    test_sentence = test_x.iloc[idx]
    
    # 특수문자 제거
    cleaned_sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", test_sentence) # [ whitespaces, 숫자, 영문 알파벳, 한글(+자모음) ]이 아닌 것을 공백으로 치환 (특수문자 제거)
    
    # Tokenizing / Tokens to sequence numbers / Padding
    encoded_dict = tokenizer.encode_plus(text=cleaned_sentence,
                                         padding='max_length', 
                                         truncation=True,
                                         max_length=SEQ_LEN) # SEQ_LEN == 128
    
    token_ids.append(encoded_dict['input_ids']) # tokens_tensor
    token_masks.append(encoded_dict['attention_mask']) # masks_tensor
    token_segments.append(encoded_dict['token_type_ids']) # segments_tensor

    test_labels.append(test_y.iloc[idx])


test_inputs = (np.array(token_ids), np.array(token_masks), np.array(token_segments))
test_labels = np.array(test_labels)

100%|██████████| 33702/33702 [00:32<00:00, 1046.63it/s]


In [28]:
model.load_weights(filepath=checkpoint_path + 'best_kobert_weights.h5') # Load the best model's weights from checkpoint file (이번 실습에서 변경되었습니다)

preds = model.predict(test_inputs)
preds = tf.argmax(preds, axis=1)

print(accuracy_score(preds, test_y)) 

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 128) dtype=int64>]


0.7417067236365794


In [29]:
drive.mount('/gdrive')


data_path = '/gdrive/MyDrive/colab_data/temp_data/' 

if not os.path.exists(data_path): 
    os.makedirs(data_path)


with open(data_path + 'model_koBERTfunction_v1.pkl', 'wb') as f: # 이번 실습에서 변경되었습니다
    pickle.dump(dill.dumps(create_model), f) # use dill to pickle a function (https://j.mp/3CeSIzP & https://j.mp/3AaXxYW)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [30]:
!pip install transformers==4.8.2
!pip install tensorflow_addons

import pandas as pd
import numpy as np

import re
import os
import pickle 
import dill # for saving a function as a file
import logging # for changing the tf's logging level
from IPython.display import clear_output 

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers, initializers, losses, optimizers, metrics 

import transformers
from transformers import TFBertModel 

# 이번 실습에서 추가되었습니다
!pip install sentencepiece==0.1.96
import sentencepiece as spm 
!git clone https://github.com/monologg/KoBERT-Transformers.git 
!mv KoBERT-Transformers/kobert_transformers/tokenization_kobert.py /content 
from tokenization_kobert import KoBertTokenizer 

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

# clear the output after the installation
clear_output() 

In [31]:
drive.mount('/gdrive')

data_path = '/gdrive/MyDrive/colab_data/temp_data/' 


# 1) Load the Model-builder function
with open(data_path + 'model_koBERTfunction_v1.pkl', 'rb') as f: # 이번 실습에서 변경되었습니다
    create_model = dill.loads(pickle.load(f)) # use dill to pickle a python function

# 2) Load the Bert-tokenizer 
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

# 3) Create the model & load the Model-weights (from checkpoint file)
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver) # Obsolete : tf.distribute.experimental.TPUStrategy()

with strategy.scope(): 
    model = create_model(max_length=128) 

checkpoint_path = '/gdrive/MyDrive/colab_data/temp_data/saved_models/'
model.load_weights(filepath=checkpoint_path + 'best_kobert_weights.h5') # 이번 실습에서 변경되었습니다

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [32]:
def predict_sentiment(sentence, tokenizer, model):
    
    SEQ_LEN = 128 # 최대 token 개수 이상의 값으로 임의로 설정

    # Tokenizing / Tokens to sequence numbers / Padding
    encoded_dict = tokenizer.encode_plus(text=re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence),
                                         padding='max_length', 
                                         truncation = True,
                                         max_length=SEQ_LEN) # SEQ_LEN == 128
    
    token_ids = np.array(encoded_dict['input_ids']).reshape(1, -1) # shape == (1, 128) : like appending to a list 
    token_masks = np.array(encoded_dict['attention_mask']).reshape(1, -1)
    token_segments = np.array(encoded_dict['token_type_ids']).reshape(1, -1)
    
    new_inputs = (token_ids, token_masks, token_segments)

    # Prediction
    prediction = model.predict(new_inputs)
    predicted_probability = np.round(np.max(prediction) * 100, 2) # ex) [[0.0125517 0.9874483]] -> round(0.9874483 * 100, 2) -> round(98.74483, 2) -> 98.74
    predicted_class = ['부정', '긍정','중립'][np.argmax(prediction, axis=1)[0]] # ex) ['부정', '긍정'][[1][0]] -> ['부정', '긍정'][1] -> '긍정'
    
    print("{}% 확률로 {} 문장입니다.".format(predicted_probability, predicted_class))

In [39]:
new_sentence = "아 오늘은 정말 힘들고 슬픈 하루였다."

predict_sentiment(new_sentence, tokenizer, model)

97.83% 확률로 부정 문장입니다.
